# Multiparty computation in pytorch demo
Model owner code

## Imports 

In [1]:
from interface.distributed_interface import DistributedInterface

In [2]:
from shared_variable import SharedVariable

In [3]:
import torch
from torch.autograd import Variable

In [4]:
import spdz

## Define an iterface for sending tensors
The interface used in this demo is the distributed interface. The distributed interface uses pytorch.distributed to send tensors. Notably this interface should only be intialized once per machine

In [5]:
interface = DistributedInterface(0)

## Recieving data from other party
Must pass swap shares tensor of the same size as the tensor being received. For this demo the values of the tensor on the receiving end do not matter as they are ignored

In [6]:
raw_data= spdz.swap_shares(torch.LongTensor(1,2).zero_(),interface)

In [7]:
data = SharedVariable(Variable(raw_data),interface)

## Define and send weights to other party
The weights here are defined as a float tensor and then encoded into a fixed point representation. This fixed point representation allows us to avoid overflow errors and allows for easier multiplication

In [8]:
raw_weights = spdz.encode(torch.FloatTensor([[2],[2]]))

After the weights are encoded they are divided up so each party gets one part of the weights. These shares add up to the true value of the weights

In [9]:
weights_self,weights_other = spdz.share(raw_weights)

In [10]:
spdz.swap_shares(weights_other,interface)


 0
 0
[torch.LongTensor of size 2x1]

In [11]:
weights = SharedVariable(Variable(weights_self,requires_grad=True),interface)

## Actual computation
The actual computation of this demo is to compute a matrix multiplication between the weights and the data. This computation in its entirety is take care of in the following cell. Adding additional computations is as simple as chaining them together

In [12]:
output = data @weights

In [13]:
output

Variable containing:
 3.3858e+08
[torch.LongTensor of size 1x1]

## Backwards pass
The backwards pass is handled by the following cell. It simply calls backward as you would with a variable. We can call this backward pass without recombining the result

In [14]:
output.backward(torch.LongTensor([[1]]))

In [15]:
weights.grad

Variable containing:
 2.3567e+08
 1.4051e+09
[torch.LongTensor of size 2x1]

## Recombining weight gradient
By simply adding the gradients together, we get the true gradient

In [16]:
weights_grad_other = spdz.swap_shares(weights.grad.data,interface)

In [17]:
weights_grad = spdz.decode(spdz.reconstruct([weights.grad.data,weights_grad_other]))

In [18]:
weights_grad


 6
 6
[torch.FloatTensor of size 2x1]

We get no gradient for data because we set it not to require a gradient

In [19]:
data.grad

## Recombining output
Finally we can recombine the output and see that the calculation was correct

In [20]:
output_other = spdz.swap_shares(output.var.data,interface)

In [21]:
output_total = spdz.decode(spdz.reconstruct([output.var.data,output_other]))

In [22]:
output_total


 12
[torch.FloatTensor of size 1x1]

In [23]:
torch.FloatTensor([[3,3]])@torch.FloatTensor([[2],[2]])


 12
[torch.FloatTensor of size 1x1]